In [1]:
sc.install_pypi_package("numpy")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1742277623694_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("NYC Taxi Fare Predicition").getOrCreate()

s3_path = "s3://nyc-taxi-fare-data/train.csv"
df = spark.read.csv(s3_path, header = True, inferSchema = True)

df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|                key|fare_amount|    pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|2009-06-15 17:26:21|        4.5|2009-06-15 17:26:21|      -73.844311|      40.721319|        -73.84161|       40.712278|              1|
|2010-01-05 16:52:16|       16.9|2010-01-05 16:52:16|      -74.016048|      40.711303|       -73.979268|       40.782004|              1|
|2011-08-18 00:35:00|        5.7|2011-08-18 00:35:00|      -73.982738|       40.76127|       -73.991242|       40.750562|              2|
|2012-04-21 04:30:42|        7.7|2012-04-21 04:30:42|       -73.98713|      40.733143|       -73.991567|       40.758092|              1|
|2010-03-09 07:51:00|        5.3|2

In [3]:
df.printSchema()
df.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- key: timestamp (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)

+-------+------------------+------------------+------------------+------------------+-----------------+------------------+
|summary|       fare_amount|  pickup_longitude|   pickup_latitude| dropoff_longitude| dropoff_latitude|   passenger_count|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+
|  count|          55423856|          55423856|          55423856|          55423480|         55423480|          55423856|
|   mean|11.345045601663854|-72.50968444358728|39.919791786888176| -72.5112097297181|39.92068144482886|1.6853799201556816|
| stddev

In [4]:
from pyspark.sql.functions import col

df_cleaned = df.dropna()
df_cleaned = df_cleaned.filter(
    (col("fare_amount") >= 2.50) &
    (col("fare_amount") <= 150)
)

df_cleaned = df_cleaned.filter(
    (col("passenger_count") >= 1) &
    (col("passenger_count") <= 5)
)

NYC_LONG_MIN, NYC_LONG_MAX = -74.5, -72.0
NYC_LAT_MIN, NYC_LAT_MAX = 40.0, 42.0

df_cleaned = df_cleaned.filter(
    (col("pickup_longitude") >= NYC_LONG_MIN) & (col("pickup_longitude") <= NYC_LONG_MAX) &
    (col("dropoff_longitude") >= NYC_LONG_MIN) & (col("dropoff_longitude") <= NYC_LONG_MAX) &
    (col("pickup_latitude") >= NYC_LAT_MIN) & (col("pickup_latitude") <= NYC_LAT_MAX) &
    (col("dropoff_latitude") >= NYC_LAT_MIN) & (col("dropoff_latitude") <= NYC_LAT_MAX) 
)

df_cleaned.show(5)
df_cleaned.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|                key|fare_amount|    pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|2009-06-15 17:26:21|        4.5|2009-06-15 17:26:21|      -73.844311|      40.721319|        -73.84161|       40.712278|              1|
|2010-01-05 16:52:16|       16.9|2010-01-05 16:52:16|      -74.016048|      40.711303|       -73.979268|       40.782004|              1|
|2011-08-18 00:35:00|        5.7|2011-08-18 00:35:00|      -73.982738|       40.76127|       -73.991242|       40.750562|              2|
|2012-04-21 04:30:42|        7.7|2012-04-21 04:30:42|       -73.98713|      40.733143|       -73.991567|       40.758092|              1|
|2010-03-09 07:51:00|        5.3|2

In [5]:
cleaned_s3_path = "s3://nyc-taxi-fare-data/cleaned-data/"
df_cleaned.write.mode("overwrite").parquet(cleaned_s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_cleaned = spark.read.parquet("s3://nyc-taxi-fare-data/cleaned-data/")
df_cleaned.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|                key|fare_amount|    pickup_datetime|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|passenger_count|
+-------------------+-----------+-------------------+----------------+---------------+-----------------+----------------+---------------+
|2014-03-28 22:33:44|       10.5|2014-03-28 22:33:44|      -73.975948|      40.760499|       -74.006053|       40.748556|              2|
|2009-06-24 10:53:42|        6.1|2009-06-24 10:53:42|      -73.965135|      40.755205|       -73.964978|       40.765663|              1|
|2013-09-21 00:41:33|       15.5|2013-09-21 00:41:33|      -73.999178|      40.727815|       -73.955996|       40.772278|              2|
|2013-10-10 02:04:00|       26.0|2013-10-10 02:04:00|      -74.013262|      40.708937|        -73.95027|       40.635287|              1|
|2014-03-15 12:36:00|       52.0|2

In [7]:
from pyspark.sql.functions import radians, cos, sin, atan2, sqrt

def haversine_distance(lon1, lat1, lon2, lat2):
    R = 6371.0
    return 2 * R * atan2(sqrt(
        sin(radians(lat2 - lat1) / 2)**2 + 
        cos (radians(lat1)) * cos(radians(lat2)) *
        sin(radians(lon2 - lon1) / 2)**2
    ),
    sqrt(
        1 - (
        sin(radians(lat2 - lat1) / 2)**2 + 
        cos (radians(lat1)) * cos(radians(lat2)) *
        sin(radians(lon2 - lon1) / 2)**2
        )
    ),
)

df_cleaned = df_cleaned.withColumn(
    "trip_distance_km", 
    haversine_distance(
        col("pickup_latitude"), col("pickup_longitude"), 
        col("dropoff_latitude"), col("dropoff_longitude")
    )
) 

df_cleaned.select("trip_distance_km").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|   trip_distance_km|
+-------------------+
| 3.3674989102143535|
|0.32168794619024293|
|  4.991768280690499|
|  7.359914633548004|
| 15.882108914214808|
+-------------------+
only showing top 5 rows

In [8]:
from pyspark.ml.feature import VectorAssembler 

feature_columns = ["trip_distance_km", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude", "passenger_count"]

assembler = VectorAssembler(inputCols = feature_columns, outputCol = "features")
df_model = assembler.transform(df_cleaned)

df_model.select("features", "fare_amount").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|            features|fare_amount|
+--------------------+-----------+
|[3.36749891021435...|       10.5|
|[0.32168794619024...|        6.1|
|[4.99176828069049...|       15.5|
|[7.35991463354800...|       26.0|
|[15.8821089142148...|       52.0|
+--------------------+-----------+
only showing top 5 rows

In [9]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol = "features", outputCol = "scaled_features", withMean = True, withStd = True)

scaler_model = scaler.fit(df_model)
df_scaled = scaler_model.transform(df_model)

df_scaled.select("scaled_features", "fare_amount").show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|     scaled_features|fare_amount|
+--------------------+-----------+
|[0.16976121913776...|       10.5|
|[-0.6075941723076...|        6.1|
|[0.58430912352612...|       15.5|
|[1.18871017036371...|       26.0|
|[3.36375443583875...|       52.0|
+--------------------+-----------+
only showing top 5 rows

In [10]:
df_sampled = df_scaled.sample(withReplacement = False, fraction = 0.3, seed = 42)
print(f"Sampled Dataset Size: {df_sampled.count()} rows")

train_data, test_data = df_sampled.randomSplit([0.7,0.3], seed = 42)
print(f"Training Data: {train_data.count()} rows")
print(f"Testing Data: {test_data.count()} rows")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Sampled Dataset Size: 15870491 rows
Training Data: 11110384 rows
Testing Data: 4760107 rows

In [11]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol = "scaled_features", labelCol = "fare_amount")
lr_model = lr.fit(train_data)
lr_predictions = lr_model.transform(test_data)
lr_predictions.select("fare_amount", "prediction").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+
|fare_amount|        prediction|
+-----------+------------------+
|       13.8|18.592860074859757|
|        4.6| 6.657464216527172|
|        5.0| 8.011823189740728|
|        4.6| 7.663537880954089|
|        7.4| 9.710376664735577|
|        4.2| 9.309216777817195|
|       16.6| 10.35327315724484|
|        7.4|  9.63320431267817|
|       14.6|11.978001259791766|
|        9.4| 8.982121950665192|
+-----------+------------------+
only showing top 10 rows

In [12]:
from pyspark.ml.evaluation import RegressionEvaluator

lr_rmse_evaluator = RegressionEvaluator(labelCol = "fare_amount", predictionCol = "prediction", metricName = "rmse")
lr_rmse = lr_rmse_evaluator.evaluate(lr_predictions)

lr_mae_evaluator = RegressionEvaluator(labelCol = "fare_amount", predictionCol = "prediction", metricName = "mae")
lr_mae = lr_mae_evaluator.evaluate(lr_predictions)

print(f"Linear Regression RMSE: {lr_rmse:.4f}")
print(f"Linear Regression MAE: {lr_mae:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Linear Regression RMSE: 5.6278
Linear Regression MAE: 3.1592

In [13]:
from pyspark.ml.regression import GBTRegressor

gbt = GBTRegressor(featuresCol = "scaled_features", labelCol = "fare_amount", maxIter = 10)
gbt_model = gbt.fit(train_data)
gbt_predictions = gbt_model.transform(test_data)
gbt_predictions.select("fare_amount", "prediction").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+
|fare_amount|        prediction|
+-----------+------------------+
|       13.8|15.577520053004926|
|        4.6| 5.694017874777113|
|        5.0| 6.616806933783921|
|        4.6| 7.667671197947389|
|        7.4| 8.694600192095036|
|        4.2| 6.698182523613864|
|       16.6|10.537640960481848|
|        7.4| 8.843612927406095|
|       14.6|14.589011894492364|
|        9.4| 7.504000844014802|
+-----------+------------------+
only showing top 10 rows

In [14]:
gbt_rmse_evaluator = RegressionEvaluator(labelCol = "fare_amount", predictionCol = "prediction", metricName = "rmse")
gbt_rmse = gbt_rmse_evaluator.evaluate(gbt_predictions)

gbt_mae_evaluator = RegressionEvaluator(labelCol = "fare_amount", predictionCol = "prediction", metricName = "mae")
gbt_mae = gbt_mae_evaluator.evaluate(gbt_predictions)

print(f"GBT RMSE: {gbt_rmse:.4f}")
print(f"GBT MAE: {gbt_mae:.4f}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

GBT RMSE: 4.6415
GBT MAE: 2.6664

In [15]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

gbt = GBTRegressor(featuresCol = "scaled_features", labelCol = "fare_amount")

paramGrid = (ParamGridBuilder()
            .addGrid(gbt.maxDepth, [3,5])
            .addGrid(gbt.maxIter, [10])
            .addGrid(gbt.stepSize, [0.1])
            .build())

evaluator = RegressionEvaluator(labelCol = "fare_amount", predictionCol = "prediction", metricName = "rmse")

trainValidationSplit = TrainValidationSplit(
    estimator = gbt,
    estimatorParamMaps = paramGrid,
    evaluator = evaluator, 
    trainRatio = 0.8
)

train_sample = train_data.sample(fraction = 0.3, seed = 42)
tvModel = trainValidationSplit.fit(train_sample)
bestModel = tvModel.bestModel

best_maxDepth = bestModel._java_obj.getMaxDepth()
best_maxIter = bestModel._java_obj.getMaxIter()
best_stepSize = bestModel._java_obj.getStepSize()

print(f"Best Hyperparameter")
print(f"Best maxDepth: {best_maxDepth}")
print(f"Best maxIter: {best_maxIter}")
print(f"Best stepSize: {best_stepSize}")

predictions = bestModel.transform(test_data)

rmse = evaluator.evaluate(predictions)
mae_evaluator = RegressionEvaluator(labelCol = "fare_amount", predictionCol = "prediction", metricName = "mae")
mae = mae_evaluator.evaluate(predictions)

print(f"Optimized GBT RMSE: {rmse:.4f}")
print(f"Optimized GBT MAE: {mae:.4f}")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Best Hyperparameter
Best maxDepth: 5
Best maxIter: 10
Best stepSize: 0.1
Optimized GBT RMSE: 4.6488
Optimized GBT MAE: 2.6694

In [16]:
predictions.select("fare_amount", "prediction").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+------------------+
|fare_amount|        prediction|
+-----------+------------------+
|       13.8|15.985803965387955|
|        4.6|5.6623125555775236|
|        5.0|6.5917231370935765|
|        4.6|  7.53779728027998|
|        7.4|  8.29401590191077|
|        4.2| 7.000574067299341|
|       16.6| 9.391705773370417|
|        7.4|  8.63509778837609|
|       14.6|14.118625646934285|
|        9.4|  7.53779728027998|
+-----------+------------------+
only showing top 10 rows